In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat 
import os
import glob

In [2]:
import pandas as pd
from tqdm.notebook import tqdm
from scipy import interpolate

In [3]:
movie_info = np.load("../preprocessed_data/clips_png/movie_info.npz", allow_pickle=True)

In [4]:
subj_data_dir = "../preprocessed_data/subj_gaze_pos_upsampled/"
dirs = os.listdir(subj_data_dir)
video_names = sorted([os.path.splitext(os.path.basename(f))[0] for f in dirs])

In [5]:
gaze_path = "../GazeData/NV/"
dirs = os.listdir(gaze_path)
subj_names = sorted([f for f in dirs if os.path.isdir(os.path.join(gaze_path, f))])

In [6]:
len(subj_names)

63

In [7]:
num_upsample = 30
gbvs_keys = ['All', 'Intensity', 'Orientation', 'Contrast', 'Flicker', 'Motion', 'DKL_Color']

In [7]:
video_names

['ADVEN_12a',
 'ADVEN_15a',
 'ADVEN_18a',
 'ADVEN_24a',
 'ADVEN_3a',
 'ADVEN_9a',
 'AMAZI_14a',
 'AMAZI_20a',
 'AMAZI_2a',
 'AMAZI_2b',
 'AMAZI_8a',
 'APPAL_10a',
 'APPAL_18a',
 'APPAL_2a',
 'APPAL_5a',
 'APPAL_7b',
 'BATM_15a',
 'BATM_18a',
 'BATM_21a',
 'BATM_37a',
 'BATM_6a',
 'BATM_9a',
 'BLIND_5a',
 'BOOKC_11a',
 'BOOKC_15a',
 'BOOKC_23a',
 'BOOKC_27a',
 'BOOKC_3a',
 'BURY_15a',
 'BURY_1b',
 'BURY_3a',
 'BURY_5b',
 'BURY_7a',
 'CORAL_11a',
 'CORAL_14a',
 'CORAL_17a',
 'CORAL_5a',
 'CORAL_5b',
 'CORAL_8a',
 'Cloud_13a',
 'Cloud_17a',
 'Cloud_17b',
 'Cloud_20a',
 'Cloud_24a',
 'Cloud_3a',
 'Cloud_6a',
 'Cloud_9a',
 'DEEPB_11a',
 'DEEPB_13a',
 'DEEPB_3a',
 'DEEPB_5a',
 'DEEPB_5b',
 'DEEPB_7a',
 'DEEPB_9a',
 'DEEPB_9b',
 'DEEPB_9c',
 'DREAMG_3a',
 'DREAMG_9a',
 'EASTE_11a',
 'EASTE_15a',
 'EASTE_19a',
 'EASTE_2a',
 'EASTE_8b',
 'FOODI_10a',
 'FOODI_2a',
 'FOODI_2b',
 'FOODI_4a',
 'FOODI_6a',
 'FOODI_8a',
 'FORGE_11a',
 'FORGE_14a',
 'FORGE_17a',
 'FORGE_8a',
 'FREED_12a',
 'FREED_14a'

In [11]:
gbvs_data_dir = "../preprocessed_data/gbvs_gaze_pos"
num_subj = len(subj_names)

In [35]:
#v = video_names[0]
dist_mat_dict = {}
for v in tqdm(video_names):
    movie_info_key = None
    for key in movie_info.files:
        if v in key:
            movie_info_key = key
    print(v, movie_info_key)
    dist_mat_dict[v] = {}
    
    # load subject gaze pos
    subj_gaze_pos = np.load(f"{subj_data_dir}/{v}.npz", allow_pickle=True)
    
    # load vit gaze pos
    gbvs_gaze_pos_data = loadmat(f"{gbvs_data_dir}/{movie_info_key}.mat", simplify_cells=True)["gbvs_res"]
    
    for gbvs_key in gbvs_keys:
        gbvs_gaze_pos_key = gbvs_gaze_pos_data[gbvs_key]
        
        # upsampling 
        gbvs_gaze_pos_key_upsampled = np.repeat(gbvs_gaze_pos_key, num_upsample, axis=0)
        dist_mat = np.zeros(num_subj)
        for i in range(num_subj):
            si = subj_names[i]
            if subj_gaze_pos.get(si) is not None:
                gaze_diff = gbvs_gaze_pos_key_upsampled - subj_gaze_pos[si].T
                diff_norm = np.linalg.norm(gaze_diff, axis=-1)
                dist_mat[i] = np.nanmedian(diff_norm, axis=-1)
            else:
                dist_mat[i] = np.nan
        dist_mat_dict[v][gbvs_key] = dist_mat

  0%|          | 0/200 [00:00<?, ?it/s]

ADVEN_12a ADVEN_12a_c
ADVEN_15a ADVEN_15a_c
ADVEN_18a ADVEN_18a_c
ADVEN_24a ADVEN_24a_c
ADVEN_3a ADVEN_3a_c
ADVEN_9a ADVEN_9a_c
AMAZI_14a AMAZI_14a_c
AMAZI_20a AMAZI_20a_c
AMAZI_2a AMAZI_2a_c
AMAZI_2b AMAZI_2b_c
AMAZI_8a AMAZI_8a_c
APPAL_10a APPAL_10a_c
APPAL_18a APPAL_18a_c
APPAL_2a APPAL_2a_c
APPAL_5a APPAL_5a_c
APPAL_7b APPAL_7b_c
BATM_15a BATM_15a_c
BATM_18a BATM_18a_c
BATM_21a BATM_21a_c
BATM_37a BATM_37a_c
BATM_6a BATM_6a_c
BATM_9a BATM_9a_c
BLIND_5a BLIND_5a_c
BOOKC_11a BOOKC_11a_c
BOOKC_15a BOOKC_15a_c
BOOKC_23a BOOKC_23a_c
BOOKC_27a BOOKC_27a_c
BOOKC_3a BOOKC_3a_c
BURY_15a BURY_15a_c
BURY_1b BURY_1b_c
BURY_3a BURY_3a_c
BURY_5b BURY_5b_c
BURY_7a BURY_7a_c
CORAL_11a CORAL_11a_c
CORAL_14a CORAL_14a_c
CORAL_17a CORAL_17a_c
CORAL_5a CORAL_5a_c
CORAL_5b CORAL_5b_c
CORAL_8a CORAL_8a_c
Cloud_13a Cloud_13a_c
Cloud_17a Cloud_17a_c
Cloud_17b Cloud_17b_c
Cloud_20a Cloud_20a_c
Cloud_24a Cloud_24a_c
Cloud_3a Cloud_3a_c
Cloud_6a Cloud_6a_c
Cloud_9a Cloud_9a_c
DEEPB_11a DEEPB_11a_c
DEEPB_13a 

In [36]:
np.savez_compressed("../preprocessed_data/subj2gbvs_dist.npz", **dist_mat_dict)

In [37]:
subj2subj_data = np.load("../preprocessed_data/subj2subj_dist.npz", allow_pickle=True)

In [38]:
dist_nanmedian = subj2subj_data["dist_nanmedian"]

In [40]:
dist_mat_mean_dict = {}
for gbvs_key in gbvs_keys:
    dist_mat_all = []
    for i, v in tqdm(enumerate(video_names)):
        dist_mat = dist_mat_dict[v][gbvs_key]
        dist_mat_normalized = dist_mat / dist_nanmedian[i]
        dist_mat_all.append(dist_mat_normalized)
    dist_mat_all = np.array(dist_mat_all)
    dist_mat_mean_dict[gbvs_key] = np.nanmean(dist_mat_all, axis=0)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [43]:
np.savez_compressed("../preprocessed_data/subj2gbvs_mean_dist.npz", **dist_mat_mean_dict)